# Is there a nematic dimer phase at $\theta=0$?

In [1]:
import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.collections as clt
import pandas as pd
import string as st
import scipy.spatial as spp
import scipy.optimize as spo
import sympy as smp
import copy as cp
import pickle

from IPython.display import HTML

smp.init_printing()
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
12*25

## Dobnikar Parameters

Dobnikar shows that they obtain a dimer minimum energy configuration for a packing fraction $\phi=0.437$ and an inter-wall separation of $h=1.445\sigma$. Here we use those parameters and run a long simulation of $60\mathrm{s}$.
This takes about $12\mathrm{min}$ to run.

In this version we use a field of $B_z = 21.48$ 

The interaction becomes attractive at 
$$(h-\sigma)>\frac{\sigma}{\sqrt{5}} \approx 0.447\sigma$$

In [3]:
angle = 0
radius = 1.4

region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=0.437, height = 4.046)

sim = lmp.sim(initial_positions,
          magnitude = 7.28*3,
          file_name = "test_2",
          dir_name = "C:/Users/aortiza/Desktop/NematicPhase",
          radius = radius,
          diffusion = 0.073,
          frequency=0,
          susceptibility = 0.4,
          angle=angle,
          framerate=1e1,
          timestep=0.5e-4,
          total_time = 300,
          temperature = 300,
          stamp_time = False,
          space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

sim.generate_scripts()
sim.run()

In [ ]:
trj = sim.load(read_trj=True)

## The full minute displayed at 18x the speed

In [9]:
HTML(lmp.display_animation_direct(sim,trj,start=0,end=300,step=18,speedup=18))

## Now we map the field amplitude to see if we find nematic dimer ordering

In [7]:
field = np.linspace(1,30,30)
field

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.])

In [13]:
angle = 0
radius = 1.4

region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=0.437, height = 4.046)

for i,f in enumerate(field):
    sim = lmp.sim(initial_positions,
                  magnitude = f,
                  file_name = "field_%u"%f,
                  dir_name = "C:/Users/aortiza/Desktop/NematicPhase/Field_a/",
                  radius = radius,
                  diffusion = 0.073,
                  frequency=0,
                  susceptibility = 0.4,
                  angle=angle,
                  framerate=1e2,
                  timestep=0.5e-4,
                  total_time = 120,
                  temperature = 300,
                  stamp_time = False,
                  space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

    sim.generate_scripts()
    sim.run()

This will take about 12 hours. 

In [15]:
field[29:]

array([ 30.])

The 30mT run failed. I will try with a smaller timestep.

In [ ]:
for i,f in enumerate(field):
    sim = lmp.sim(initial_positions,
                  magnitude = f,
                  file_name = "field_%u"%f,
                  dir_name = "C:/Users/aortiza/Desktop/NematicPhase/Field_a/",
                  radius = radius,
                  diffusion = 0.073,
                  frequency=0,
                  susceptibility = 0.4,
                  angle=angle,
                  framerate=1e2,
                  timestep=0.25e-4,
                  total_time = 120,
                  temperature = 120,
                  stamp_time = False,
                  space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

    sim.generate_scripts()
    sim.run()

# Conditions where Helena found dimers

In [9]:
angle = 0
radius = 1.4

region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=0.17, height = 4.2)

sim = lmp.sim(initial_positions,
          magnitude = 6.5,
          file_name = "test_2",
          dir_name = "C:/Users/aortiza/Desktop/NematicPhase",
          radius = radius,
          diffusion = 0.073,
          frequency=0,
          susceptibility = 0.4,
          angle=angle,
          framerate=1e1,
          timestep=0.5e-4,
          total_time = 300,
          temperature = 300,
          stamp_time = False,
          space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

sim.generate_scripts()
sim.run()

trj = lmp.trj_lazyread(sim.base_name+".lammpstrj")[::100]
lmp.export_animation(sim,trj,verb=False,start=0,end=sim.run_parameters.total_time,step=1,speedup=100)

In [10]:
HTML(lmp.display_animation_direct(sim,trj,start=0,end=300,step=1,speedup=100))

In [14]:
angle = 0
radius = 1.4

region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=0.17, height = 4.2)

sim = lmp.sim(initial_positions,
              magnitude = 0,
              file_name = "test_1",
              dir_name = "/home/aortiza/NematicPhase/",
              radius = radius,
              diffusion = 0.073,
              frequency=0,
              susceptibility = 0.4,
              angle=angle,
              framerate=1e0,
              timestep=1e-4,
              total_time = 30,
              temperature = 300,
              stamp_time = False,
              space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

sim.generate_scripts()
sim.run()

trj = lmp.trj_lazyread(sim.base_name+".lammpstrj")[-1]
initial_positions = trj.filter(('x','y','z')).values

sim = lmp.sim(initial_positions,
          magnitude = 6.5,
          file_name = "test_1",
          dir_name = "C:/Users/aortiza/Desktop/NematicPhase",
          radius = radius,
          diffusion = 0.073,
          frequency=0,
          susceptibility = 0.4,
          angle=angle,
          framerate=1e1,
          timestep=0.5e-4,
          total_time = 300,
          temperature = 300,
          stamp_time = False,
          space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

sim.generate_scripts()
sim.run()

trj = lmp.trj_lazyread(sim.base_name+".lammpstrj")[::100]
lmp.export_animation(sim,trj,verb=False,start=0,end=sim.run_parameters.total_time,step=1,speedup=100)

In [15]:
HTML(lmp.display_animation_direct(sim,trj,start=0,end=300,step=1,speedup=100))

In [16]:
trj = lmp.trj_lazyread(sim.base_name+".lammpstrj")[:30]

In [17]:
HTML(lmp.display_animation_direct(sim,trj,start=0,end=3,step=1,speedup=1))

# Some other heights and packing fractions 
## Parallel runing of dimer simulations in several cores

The rest of this notebook shows the method used to run a sweep of parameters in several cores at the same time. Unfortunatelly, there is an error in the parameters, and all the results have a temperature of 120K. However, this commands are used in subsequent notebooks to obtain faster results. The results are saved in 
    
    MAGNETIC\Antonio\SimulationsOfDimerSystem\NematicPhase\ParameterSpace_120K\

In [29]:
from IPython.display import clear_output

In [3]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [25]:
%px import os; os.chdir('D:\\Dropbox\\Colloids\\SimulationsOfDimerSystem')

In [35]:
%%px 
import lammps2d as lmp
import numpy as np
import pickle
import datetime
import time

In [39]:
height = [3.8,3.9,4,4.1,4.2]
packing = [0.4,0.42,0.44,0.46,0.48,0.5]
field = np.linspace(1,20,20)

H,P,F = np.meshgrid(height,packing,field)

height = H.flatten()
packing = P.flatten()
field = F.flatten()

In [40]:
%%px

height = [3.8,3.9,4,4.1,4.2]
packing = [0.4,0.42,0.44,0.46,0.48,0.5]
field = np.linspace(1,20,20)

H,P,F = np.meshgrid(height,packing,field)

height = H.flatten()
packing = P.flatten()
field = F.flatten()

In [41]:
%%px
def run_sim(i):
    angle = 0
    radius = 1.4
    region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=packing[i], height = height[i])

    sim = lmp.sim(initial_positions,
                  magnitude = field[i],
                  file_name = "field_%u"%field[i],
                  dir_name = "C:/Users/aortiza/Desktop/NematicPhase/H_%g_p_%g/"%(height[i],packing[i]),
                  radius = radius,
                  diffusion = 0.073,
                  frequency=0,
                  susceptibility = 0.4,
                  angle=angle,
                  framerate=1e2,
                  timestep=0.5e-4,
                  total_time = 120,
                  temperature = 120,
                  stamp_time = False,
                  space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})
    
    """ Has the simulation been run to completion?"""
    filename = os.path.join(sim.sim_parameters.dir_name,sim.sim_parameters.file_name)
    
    if os.path.isfile(filename+".p") and os.path.isfile(filename+".lammpstrj"):
        
        os.path.isfile(filename+".lammpstrj")
        lz_rd = lmp.trj_lazyread(filename+".lammpstrj")
        last_frame_recorded = max(list(lz_rd.T.keys()))

        sim_p = pickle.load(open(filename+".p", "rb" ))
        last_frame_expected = sim_p.run_parameters.total_time/sim_p.run_parameters.timestep
        
        if last_frame_expected == last_frame_recorded:
            print(filename," was done")
            return
        
    sim.generate_scripts()
    sim.run()
    print(filename," is done at " + str(datetime.datetime.now()))

In [42]:
i = range(len(height))

In [ ]:
amr = dview.map_async(lambda x: run_sim(x), i[:])
while not amr.wait(1):
    # clear output because we are printing the whole thing each time
    # rather than keeping track of what we've consumed
    clear_output(wait=True)
    for stdout in amr.stdout:
        sys.stdout.write(stdout)

In [51]:
amr.started

[datetime.datetime(2018, 3, 13, 10, 6, 18, 271344, tzinfo=tzutc()),
 datetime.datetime(2018, 3, 13, 10, 6, 18, 271344, tzinfo=tzutc()),
 datetime.datetime(2018, 3, 13, 10, 6, 18, 271344, tzinfo=tzutc()),
 datetime.datetime(2018, 3, 13, 10, 6, 18, 271344, tzinfo=tzutc())]